<a href="https://colab.research.google.com/github/oktaviacitra/classification/blob/main/Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_iris

iris = load_iris()
X, y = iris.data, iris.target

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

pipeline = Pipeline(
    [
        ('scaler', StandardScaler()), 
        ('model', DecisionTreeClassifier(random_state=21))
    ]
)

In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('scaler', StandardScaler()),
  ('model', DecisionTreeClassifier(random_state=21))],
 'verbose': False,
 'scaler': StandardScaler(),
 'model': DecisionTreeClassifier(random_state=21),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'model__ccp_alpha': 0.0,
 'model__class_weight': None,
 'model__criterion': 'gini',
 'model__max_depth': None,
 'model__max_features': None,
 'model__max_leaf_nodes': None,
 'model__min_impurity_decrease': 0.0,
 'model__min_samples_leaf': 1,
 'model__min_samples_split': 2,
 'model__min_weight_fraction_leaf': 0.0,
 'model__random_state': 21,
 'model__splitter': 'best'}